In [38]:
import sys
sys.path.append('../') 
%load_ext autoreload
%autoreload 2
import sklearn
import copy
import numpy as np

import seaborn as sns
sns.set()

import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
# from viz import viz
from bokeh.plotting import figure, show, output_notebook, output_file, save
from functions import merge_data
from sklearn.model_selection import RandomizedSearchCV
import load_data
import exponential_modeling


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import fit_and_predict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
df = load_data.load_county_level(data_dir = '../data/')
df = df.sort_values('#Deaths_4/3/20', ascending=False)
# outcome_cases = load_data.outcome_cases # most recent day
# outcome_deaths = load_data.outcome_deaths
important_vars = load_data.important_keys(df)
very_important_vars = ['PopulationDensityperSqMile2010',
#                        'MedicareEnrollment,AgedTot2017',
                       'PopulationEstimate2018',
                       '#ICU_beds',
                       'MedianAge2010',
                       'Smokers_Percentage',
                       'DiabetesPercentage',
                       'HeartDiseaseMortality',
                        '#Hospitals'
#                        'PopMale60-642010',
#                         'PopFmle60-642010',
#                          'PopMale65-742010',
#                          'PopFmle65-742010',
#                          'PopMale75-842010',
#                          'PopFmle75-842010',
#                          'PopMale>842010',
#                          'PopFmle>842010'
                      ]

In [128]:
max_cases = [max(v) for v in df['cases']]
df['max_cases'] = max_cases
df =  df[df['max_cases'] > 0]

In [106]:
exponential = {'model_type':'exponential'}
shared_exponential = {'model_type':'shared_exponential'}
demographics = {'model_type':'shared_exponential', 'demographic_vars':very_important_vars}
linear = {'model_type':'linear'}

## predictions we save: four separate models, simple ensemble model with exponential and shared_exponential , ensemble with all possible combinations of 3 models, full ensemble

In [129]:
all_methods = [exponential, 
               shared_exponential,
               demographics,
               linear]
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods,
                                              output_key=f'predicted_deaths_ensemble_all'
                                              )
df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods[:2],
                                              output_key=f'predicted_deaths_simple_ensemble_all'
                                              )
for (i, model) in enumerate(all_methods):
        
    if 'demographic_vars' in model:
        demographic_vars = model['demographic_vars']
    else:
        demographic_vars = []
    
    method = model['model_type']
    #print(method)
    method_name = model['model_type']
    if 'demographic_vars' in model:
        method_name = 'demographics'
    df = fit_and_predict.fit_and_predict(df, 
                                         outcome='deaths', 
                                         method=method, 
                                         mode='predict_future', 
                                         target_day=np.array(range(1, 8)),
                                         output_key=f'predicted_deaths_{method_name}_all',
                                         demographic_vars=demographic_vars)
    print(all_methods[0:i]+all_methods[(i+1):])
    df = fit_and_predict.fit_and_predict_ensemble(df, 
                                              target_day=np.array(range(1, 8)),
                                              mode='predict_future',
                                              outcome='deaths',
                                              methods=all_methods[0:i]+all_methods[(i+1):],
                                              output_key=f'predicted_deaths_ensemble_no_{method_name}_all'
                                              )

Feature weights
log(deaths) : 0.9729621111034199
bias : 1.0854928588754749
Feature weights
PopulationDensityperSqMile2010 : 0.013618608520036173
PopulationEstimate2018 : -0.06332938122780254
#ICU_beds : 0.002601239880272427
MedianAge2010 : 0.0387013248651016
Smokers_Percentage : -0.03139838074841138
DiabetesPercentage : 0.026485570172165237
HeartDiseaseMortality : 0.050134867912497785
#Hospitals : 0.07129453913058886
log(deaths) : 0.9717954989412734
bias : 0.9591318517475634
Feature weights
log(deaths) : 0.8886066305940084
bias : 1.0682324016867693
Feature weights
log(deaths) : 0.8652206315709574
bias : 1.1020079979512758
Feature weights
log(deaths) : 0.8436428072126232
bias : 1.1082925115470559
Feature weights
log(deaths) : 0.8287462490173874
bias : 1.1162446169873024
Feature weights
log(deaths) : 0.8438141326907082
bias : 1.0982336851431942
Feature weights
log(deaths) : 0.9394081093352544
bias : 0.9539710175152932
Feature weights
log(deaths) : 0.9776441702851446
bias : 0.977803717998

Feature weights
PopulationDensityperSqMile2010 : 0.23925173186889667
PopulationEstimate2018 : 0.019894042780604384
#ICU_beds : -0.2800684889926972
MedianAge2010 : 0.20954549623023
Smokers_Percentage : 0.074984342470243
DiabetesPercentage : 0.2533674747979421
HeartDiseaseMortality : -0.0902744366356948
#Hospitals : 0.37719320181714233
log(deaths) : 0.9912346787248576
bias : 0.061098194731500244
Feature weights
PopulationDensityperSqMile2010 : 0.3270994685371543
PopulationEstimate2018 : 0.48997056114671517
#ICU_beds : -0.37617653590900973
MedianAge2010 : 0.40597680114433
Smokers_Percentage : 0.30061164895334985
DiabetesPercentage : 0.47465671485885425
HeartDiseaseMortality : -0.2962659442093009
#Hospitals : 0.16522396115849103
log(deaths) : 0.9866900396670835
bias : -0.5522202817851641
--- Model Contributions ---
{'model_type': 'linear'}: 0.8122337170518842
{'model_type': 'shared_exponential'}: 0.09794732469212634
{'model_type': 'shared_exponential', 'demographic_vars': ['PopulationDensi

Feature weights
PopulationDensityperSqMile2010 : 0.20692430221710523
PopulationEstimate2018 : -0.16669796314313448
#ICU_beds : -0.35166643197933706
MedianAge2010 : 0.06541241701979986
Smokers_Percentage : -0.14774438084924255
DiabetesPercentage : 0.1438916576480655
HeartDiseaseMortality : 0.05409871225806417
#Hospitals : 0.536272821349417
log(deaths) : 0.8173552328453046
bias : 0.7548062054985815
Feature weights
PopulationDensityperSqMile2010 : 0.2540455101507711
PopulationEstimate2018 : -0.09705112063914896
#ICU_beds : -0.36834423277796
MedianAge2010 : 0.1054905504028
Smokers_Percentage : -0.20032099107051693
DiabetesPercentage : 0.23604199320208857
HeartDiseaseMortality : 0.06747803237999474
#Hospitals : 0.48520894198505987
log(deaths) : 0.8367049806240959
bias : 0.5803654430383222
Feature weights
PopulationDensityperSqMile2010 : 0.5270512163235085
PopulationEstimate2018 : 0.22385768240333095
#ICU_beds : -0.725092738008203
MedianAge2010 : 0.16094149425231827
Smokers_Percentage : -0.3

In [130]:
method_keys = [c for c in df if 'predicted' in c]

In [131]:
for key in method_keys:
    for d in range(1, 8):
        newkey = key[:-3] + str(d)
        df[newkey] = np.array([p[d-1] for p in df[key].values])

In [132]:
df['predicted_deaths_demographics_7']

1841    2377.629037
1824    2122.454674
1803    2601.288356
1285    1781.162368
1831    1067.581540
           ...     
2510      13.209253
2491      10.549340
2489      11.618347
2462       6.735155
2473      11.802959
Name: predicted_deaths_demographics_7, Length: 2265, dtype: float64

In [133]:
[c for c in df if 'predicted' in c]

['predicted_deaths_ensemble_all',
 'predicted_deaths_simple_ensemble_all',
 'predicted_deaths_exponential_all',
 'predicted_deaths_ensemble_no_exponential_all',
 'predicted_deaths_shared_exponential_all',
 'predicted_deaths_ensemble_no_shared_exponential_all',
 'predicted_deaths_demographics_all',
 'predicted_deaths_ensemble_no_demographics_all',
 'predicted_deaths_linear_all',
 'predicted_deaths_ensemble_no_linear_all',
 'predicted_deaths_ensemble_1',
 'predicted_deaths_ensemble_2',
 'predicted_deaths_ensemble_3',
 'predicted_deaths_ensemble_4',
 'predicted_deaths_ensemble_5',
 'predicted_deaths_ensemble_6',
 'predicted_deaths_ensemble_7',
 'predicted_deaths_simple_ensemble_1',
 'predicted_deaths_simple_ensemble_2',
 'predicted_deaths_simple_ensemble_3',
 'predicted_deaths_simple_ensemble_4',
 'predicted_deaths_simple_ensemble_5',
 'predicted_deaths_simple_ensemble_6',
 'predicted_deaths_simple_ensemble_7',
 'predicted_deaths_exponential_1',
 'predicted_deaths_exponential_2',
 'predic

In [134]:
for c in df:
    if 'predicted' in c and 'all' not in c:
        print((c, df[c].max()))

('predicted_deaths_ensemble_1', 600.1626404627608)
('predicted_deaths_ensemble_2', 717.1681474112377)
('predicted_deaths_ensemble_3', 859.1548541185729)
('predicted_deaths_ensemble_4', 1033.4723728285005)
('predicted_deaths_ensemble_5', 1249.8793586057272)
('predicted_deaths_ensemble_6', 1521.413839686602)
('predicted_deaths_ensemble_7', 1865.606453153696)
('predicted_deaths_simple_ensemble_1', 615.2008631151284)
('predicted_deaths_simple_ensemble_2', 755.2049269546909)
('predicted_deaths_simple_ensemble_3', 931.1025676202316)
('predicted_deaths_simple_ensemble_4', 1153.299625092355)
('predicted_deaths_simple_ensemble_5', 1435.5925465127027)
('predicted_deaths_simple_ensemble_6', 1796.396872531156)
('predicted_deaths_simple_ensemble_7', 2260.464361090524)
('predicted_deaths_exponential_1', 609.6170653650955)
('predicted_deaths_exponential_2', 723.1147939330539)
('predicted_deaths_exponential_3', 857.7433850079061)
('predicted_deaths_exponential_4', 1017.4369556501358)
('predicted_death

In [135]:
method_keys = [c for c in df if 'predicted' in c]
geo = ['countyFIPS', 'CountyNamew/StateAbbrev']
preds_df = df[method_keys + geo]
preds_df.to_csv("../predictions/predictions_04_03.csv")

In [102]:
df['deaths'].values[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,  21,  40,  60,  92, 124,
       149, 216, 253, 305])